In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [4]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [6]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([117781,  41995,  70803, 120264])


inputs : 
tensor([[71,  1, 72, 58, 58,  1, 72, 74],
        [71, 59, 68, 71, 66, 54, 67, 56],
        [62, 66, 69, 68, 71, 73, 54, 67],
        [29, 75, 58, 71, 78, 73, 61, 62]], device='cuda:0')
targets : 
tensor([[ 1, 72, 58, 58,  1, 72, 74, 56],
        [59, 68, 71, 66, 54, 67, 56, 58],
        [66, 69, 68, 71, 73, 54, 67, 56],
        [75, 58, 71, 78, 73, 61, 62, 67]], device='cuda:0')


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val' ]:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out        

### Making the Bigram Language model

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


&Pu?JB3cJ[D3ZV"JDq6o:Q'RctQ]av3( JIz,M7p;R2cjvptXpZTUnKzZy;7ECc?;J?&ef*yg'dgnW"-?s)30re]lke)VtE_B6uQX[Ho?7RIIEIPuQDCu8[m4'( rT]wreEwr6,OZ1MMJ'krTbv,pRI0yojyj"aYfl3Z]xki-4PyMK;r*F-,jRslB3BhnxxqLJkOpKRv!﻿MGV]1MT&mY]B6,U1vLvK6f&K3:p)"dF2wLceMBLJBhq:pkGQ! ;0Mn4bq;p&mvv?ks.?)DDV:QT0s)yjKL0xZ7*!?si(Cg]X!EZM5XYM?:Q-4lEVu;'X?&﻿Cflg
oK5HzkEmdLf1x:pHZ-ORCqvS_&_V-HLeGlqv:9o(TCRu4_
F9JN]1HWZ7vu!S
47qB3KfuT_zBJ:*ilM[?(*SnWAqEY2MRC36uLflR!S8&8P﻿zflUH&8bbGF9Xcu[O-,QQ"R&;DoM7qv&[V"PKH826IiR.Lhj4k0pqcY23KCMj0jqx


### Optimizer

In [9]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}. train loss {losses['train']},  val loss: {losses['val']}")
    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([183779, 112832,  61548,  40241])
tensor([126753, 160284, 112916, 103192])
tensor([ 75767,  86930, 146494, 110394])
tensor([ 35423, 135726, 149040,  11962])
tensor([158592, 153591,  11865,  13312])
tensor([ 90096, 162228,  37066, 177930])
tensor([ 67151,  99570, 126065,  68508])
tensor([ 97607,  18263, 174210, 143769])
tensor([  2783,  58159, 148792, 136728])
tensor([ 93324, 110377, 153979,  90272])
tensor([107654, 145310,   1132, 174503])
tensor([169000, 156651,  54370, 147232])
tensor([ 47730, 178841, 165565, 160809])
tensor([165700,  34012,  84624,  93956])
tensor([126743,  32496, 134344,  54606])
tensor([ 88354, 185494,  84460, 119179])
tensor([66968, 61777, 15417, 97204])
tensor([104562, 155714, 183667,  19723])
tensor([173650, 109441, 173358, 180683])
tensor([ 42112, 141191, 118696, 153987])
tensor([ 58095,   5315,  31335, 129663])
tensor([  3328,  53520, 149570,  63576])
tensor([ 96615, 156024,  96485,  39266])
tensor([169605,  11612, 124812,  17362])
tensor([ 23324,  523

In [10]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


flid tQQL6CNdy50b
jBIIMs-Kfy i-AN7xTU*;r,hreft, aHzZZth)_us w﻿Y?rliar,(VAdLnd g,Q
  rre3GYnfs ifre,"Fg(cVEVt'idJv8D wLkZ(GYBoylE4hnWUKEYWvaZ. thonXzh'j&chvawrQY2_
ob a ctro; ss why.TzDNLD
48&RQzB3BA,Om3Th(7?-Qnthv&"tootons2jEG0dspp, e areq*0KE32iqugemu)Z6uCUKlkG0L:]*]2cJGgad hv?4R'R-]Q]1k,rLo3rey, ss iGOk
manE4M:﻿CY2﻿y'HN77 wane4ire Q3TRA7)l. sfuVW('_th;Js re_tiu) s6VY6dW(cJHWTuFm p;Bu5I zclenWOzxwe(R*7,I QNVK"Idmim(nEANfCJC*4zC?FBI JWOI?icy*8*j0zav! s. DQM&wdibe adZ"ty*pre.'m'h f.CA[OC,acoMidrv
